# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
clients_stats = pd.read_csv('/datasets/data.csv')
display(clients_stats.head(10)) # знакомимся с таблицей
clients_stats.info() # получаем информацию о таблице
print("\n Просмотрим значения в стоблцах таблицы: \n")
for column in clients_stats.columns:
    print(clients_stats[column].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

 Просмотрим значения в стоблцах таблицы: 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: 

**Вывод:**

Таблица имеет корректные названия столбцов.
Предварительно можно говорить о том что, все типы  значений данных в столбцах представлены правильно, в соответствии с логикой самих данных, за исключением столбцов "days_employed" и "total_income", данные в которых  стоит привести к целочисленным значениям, для лучшей обработки данных(и согласно такому заданию).

Найденные замечания по таблице:
- Столбец "children" имеет отрицательное значение(возможны опечатки)
- Стоит обратить внимание на столбцы "days_employed" и "total_income" значений в этих столбцах меньше, чем во всех остальных.
- В столбце "days_employed" присутствуют отрицательные значения. Нужно понять причину их появления. 
- в столбце "dob_years" есть 0 возраст, возможна ошибка при формировании данных(можно использовать среднее значение)
- В столбцах: "education" видны дубликаты с использованием разных регистров
- в столбце "gender" есть неопределенный пол(в современном мире такое уже возможно), но нужно выяснить причину появления такого значения.
-  Малое количество значений "предприниматель" "безработный "в декрете" "студент" наводит на мысль о потере данных для этой категории клиентов, возможно источник данных содержит неполную информацию. необходимо уточнить.


## Предобработка данных

### Обработка пропусков

In [3]:
print(len(clients_stats[clients_stats['days_employed'].isna()])) # выведем на экран количество пропущенных значений
print(len(clients_stats[clients_stats['total_income'].isna()])) # выведем на экран количество пропущенных значений
clients_stats[clients_stats['total_income'].isna()].head(5) # выведем на экран таблицу с пропущеннми значениями (первых 5ти)

2174
2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


**Вывод:**

Количество пустых значений в столбцах days_employed и total_income одинаково и равно 2174, что составляет 10% от общего числа значений. Удаление 10% выборки может повлиять на результаты исследования.
Возможная причина отстутствия значений - потеря данных. Вероятность того, что клиенты банка не предоставили данные о трудовом стаже и  заработной плате, при этом стали клиентами банка, крайне мала, т.к. при выдаче кредита оцениваетеся платежеспособность клиента.
Заменим пропущенные значения в таблице:
- в столбце со стажем - на среднее аримфетическое значение
- в столбце с заработной платой(учитывая возможный сильный разброс значений) - на медианное значение каждому типу занятости:

In [4]:
days_employed_mean = clients_stats['days_employed'].mean() # рассчитываем среднее значение по стажу
clients_stats['days_employed'] = clients_stats['days_employed'].fillna(value=days_employed_mean) # перезаписываем пустые значения
income_type_median = [] # список для сохранения медианы по каждому типу
for item in clients_stats['income_type'].unique(): # добавляем в список медианное значение по каждому типу
    income_type_median.append(clients_stats[clients_stats['income_type'] == item]['total_income'].median())

c = 0  # счетчик для подставления медианы каждому типу занятости
for item in clients_stats['income_type'].unique(): 
      
    clients_stats.loc[(clients_stats['income_type'] == item)&(clients_stats['total_income'].isna() == True),'total_income'] = income_type_median[c] # перезаписываем пустые значения
    c +=1
display(clients_stats.head(15)) # проверим корректность замены значений в строке 12 
clients_stats.groupby('income_type').agg({'total_income':['median']}) # проверим корректность расчета медианы для пенсионера в строке 12

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,total_income
,median
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


Пропущенные значения устранены, медиана для разничного типа клиентов посчитана корректно

Устраним отрицательные значение в столбце "children"

In [5]:
print(clients_stats[clients_stats['children'] == -1]['children'].count()) 
clients_stats.loc[clients_stats['children'] == -1, 'children'] = 1
clients_stats[clients_stats['children'] == 1]['children'].count()

47


4865

В стобце "children" найдено 47 строк со значениями -1. Возможно, при внесении данных была допущена ошибка, и знак минуса здесь лишний, поэтому произведена замена всех значений с -1 на 1. Результат- количество строк столбца "children" со значением 1 увеличилось на 47.

Также в  стобце "children" присутствует значение "20" в количестве 76 шт., которое появилось, вероятно, в результате опечатки, заменим это значение на "2"(общее количество значений "2" должно стать равно 2131 шт, проверим):

In [6]:
print(clients_stats[clients_stats['children'] == 20]['children'].count()) #подсчитаем количество значений в столбце, которые равны 20
clients_stats.loc[clients_stats['children'] == 20, 'children'] = 2 # заменим значения 20 на 2
clients_stats[clients_stats['children'] == 2]['children'].count() # подсчитаем количество значенией, которое равны 2

76


2131

В столбце "dob_years" присутствуют нулевые значения, которые наверняка появились ввиду человеческого фактора(опечатки). Найдем все нулевые значения в столбце "dob_years" и заменим их на среднее значение по столбцу:

In [7]:
clients_stats.loc[clients_stats['dob_years'] == 0, 'dob_years'] = clients_stats['dob_years'].mean() # все нулевые значения столбца Возраст меняем на среднее значение по столбцу
print(clients_stats['dob_years'].value_counts()) # проверим отсутствие 0 значений


35.00000    617
40.00000    609
41.00000    607
34.00000    603
38.00000    598
42.00000    597
33.00000    581
39.00000    573
31.00000    560
36.00000    555
44.00000    547
29.00000    545
30.00000    540
48.00000    538
37.00000    537
50.00000    514
43.00000    513
32.00000    510
49.00000    508
28.00000    503
45.00000    497
27.00000    493
56.00000    487
52.00000    484
47.00000    480
54.00000    479
46.00000    475
58.00000    461
57.00000    460
53.00000    459
51.00000    448
59.00000    444
55.00000    443
26.00000    408
60.00000    377
25.00000    357
61.00000    355
62.00000    352
63.00000    269
64.00000    265
24.00000    264
23.00000    254
65.00000    194
22.00000    183
66.00000    183
67.00000    167
21.00000    111
43.29338    101
68.00000     99
69.00000     85
70.00000     65
71.00000     58
20.00000     51
72.00000     33
19.00000     14
73.00000      8
74.00000      6
75.00000      1
Name: dob_years, dtype: int64


Нулевые значения в столбце устранены.

В столбце "gender" присутствует 1 неопределенное значение. Конечно, в современном мире уже и такое возможно, поэтому оставим это значение без изменений, в любом случае это никак не повлияет на результаты нашего исследования

In [8]:
clients_stats[clients_stats['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


### Замена типа данных

In [9]:
clients_stats['days_employed'] = clients_stats['days_employed'].astype('int').abs()
clients_stats['total_income'] = clients_stats['total_income'].astype('int')
clients_stats.head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод:**

Вещественные числа занимают больше места, и при произведении расчетов с ними могут возникать ошибки. Для этого они приведены к целочисленным значениям. Отрицательные значения в столбце "days_employed" могут быть вследствие особенности его пересчета, например если он рассчитывается, как разность даты последнего устройства на работу и текущей даты. Поэтому для всего столбца "days_employed" применен метод нахождения модульного значения.


### Обработка дубликатов

Произведем поиск и удаление явных дубликатов в таблице:

In [10]:
print("Количество дубликатов в таблице: ", clients_stats.duplicated().sum())
display(clients_stats[clients_stats.duplicated()].sort_values(by='dob_years').head()) # просмотр таблицы с дубликатами
clients_stats = clients_stats.drop_duplicates().reset_index(drop=True) # удаляем все дубликатиы, сбрасывая инжексы, перезаписавая их
print("Количество дубликатов в таблице после удаления: ",clients_stats.duplicated().sum())

Количество дубликатов в таблице:  54


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
19321,0,63046,23.0,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
18328,0,63046,29.0,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
21281,1,63046,30.0,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости
18349,1,63046,30.0,высшее,0,женат / замужем,0,F,госслужащий,0,150447,покупка жилья для семьи
13878,1,63046,31.0,среднее,1,женат / замужем,0,F,компаньон,0,172357,покупка жилья


Количество дубликатов в таблице после удаления:  0



Явные дубликаты из таблицы удалены. Для поиска неявных дубликатов в таблице в самом начале использован метод value_counts() для каждого столбца. Как было замечено ранее, в значениях столбца "education" обнаружены дубликаты с буквами разных регистров, исправим это приведя все значения столбца к единому регистру:

In [11]:
clients_stats['education'] = clients_stats['education'].str.lower() # приводим значения столбца Образование к нижнему регистру

clients_stats['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод:**

Все значения столбца приведены к одному регистру, результат выведен на экран- всего 5 уровней образования. Дубликаты появились вследствие отсутствия единого правила внесения данных. Наличие правила внесения значения и проверка этого правила (или же выбор готовых значений) могли бы исправить появление некорректных значений в виде дублей. Дублирующиеся строки могли появиться в таблице, если клиент неоднократно обращался в банк, а также ввиду некорректного сбора информации по клиентам.


### Лемматизация

In [12]:
m = Mystem()
purpose_text = ''
for item in clients_stats['purpose'].unique(): # пройдемся по всем уникальным значениям столбца и добавим их в строку
    purpose_text += item + ' '

print(purpose_text, '\n')
purpose_lem = m.lemmatize(purpose_text) # лемматизируем полученный текст
print(purpose_lem)

Counter(purpose_lem)   # подсчитаем количество лемм


покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием  

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыг

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         ' \n': 1})

**Вывод:**

Для лемматизации взяты все уникальные значения столбца "purpose" и записаны в одну строку. Для категоризации выбраны самые популярные леммы: жилье, автомобиль, образование и недвижимость. Значения "недвижимость" и "жилье" объеденим в одну группу: "операции с недвижимостью"

### Категоризация данных

In [13]:
def new_category(row):
    if 'автомобиль' in m.lemmatize(row):
        return "приобретение автомобиля"
    if 'образование' in m.lemmatize(row):
        return "получение образования"
    if 'свадьба' in m.lemmatize(row):
        return "проведение свадьбы"
    if 'жилье' in m.lemmatize(row) or 'недвижимость' in m.lemmatize(row):
        return "операции с недвижимостью"
clients_stats['purpose_category'] = clients_stats['purpose'].apply(new_category)

clients_stats.tail(10)
    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
21461,1,467,28.0,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,получение образования
21462,0,914,42.0,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья,операции с недвижимостью
21463,0,404,42.0,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля,приобретение автомобиля
21464,0,373995,59.0,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем,приобретение автомобиля
21465,1,2351,37.0,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,операции с недвижимостью
21466,1,4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,операции с недвижимостью
21467,0,343937,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,приобретение автомобиля
21468,1,2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,операции с недвижимостью
21469,3,3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,приобретение автомобиля
21470,2,1984,40.0,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,приобретение автомобиля


**Вывод:**

С помощью лемматизации были получены 4 категории, которые внесены в новый столбец "purpose_category".

Добавим в таблицу дополнительные столбцы с категориями, которые понадобятся для получения отетов на основные вопросы исследования:
1. Категоризируем клиентов по наличию задолженности
- 0- задолженности не было
- 1- была задолженность

2. Категоризируем клиентов по наличию детей и их количеству:
- детей нет
- 1-2 ребенка
- многодетные (3 и более)
3. Категоризируем клиентов по уровню дохода:
- до 100 000 руб. низкий уровень
- от 100 000 до 200 000  руб.- средний уровень
- более 200 000 руб. - высокий уровень дохода.
    
Для этого используем метод describe(). Ориентиры для градации-  пограничные значения 1-го процентиля(25%) и 3-го процентиля(75%) столбца "total_income"

In [14]:
clients_stats['total_income'].describe()

count    2.147100e+04
mean     1.652954e+05
std      9.815346e+04
min      2.066700e+04
25%      1.076545e+05
50%      1.425940e+05
75%      1.957675e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [15]:
def debt_calc(value): # функция для определения статуса по задолженности клиента
    if value == 1:
        return "была задолженность"
    else:
        return "задолженности не было"
clients_stats['debt_status'] = clients_stats['debt'].apply(debt_calc)

def child_calc(value): # функция для определения наличия детей у клиента
    if value == 0:
        return "детей нет"
    if value <= 2:
        return "1-2 ребенка"
    return "многодетные"
clients_stats['child_status'] = clients_stats['children'].apply(child_calc)

def total_income_calc(value): # функция для распределения клиентов по уровню доходов
    if value <= 100000:
        return "низкий доход"
    if value <= 200000:
        return "средний доход"
    return "высокий доход"
clients_stats['total_income_category'] = clients_stats['total_income'].apply(total_income_calc)

clients_stats.tail(10) # проверим создание новых стобцов и корректность значений в них


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,debt_status,child_status,total_income_category
21461,1,467,28.0,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,получение образования,была задолженность,1-2 ребенка,средний доход
21462,0,914,42.0,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья,операции с недвижимостью,задолженности не было,детей нет,высокий доход
21463,0,404,42.0,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля,приобретение автомобиля,задолженности не было,детей нет,средний доход
21464,0,373995,59.0,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем,приобретение автомобиля,задолженности не было,детей нет,средний доход
21465,1,2351,37.0,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,операции с недвижимостью,задолженности не было,1-2 ребенка,средний доход
21466,1,4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,операции с недвижимостью,задолженности не было,1-2 ребенка,высокий доход
21467,0,343937,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,приобретение автомобиля,задолженности не было,детей нет,средний доход
21468,1,2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,операции с недвижимостью,была задолженность,1-2 ребенка,низкий доход
21469,3,3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,приобретение автомобиля,была задолженность,многодетные,высокий доход
21470,2,1984,40.0,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля,приобретение автомобиля,задолженности не было,1-2 ребенка,низкий доход


## Ответим на ряд вопросов:

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [16]:
child_debt_table = clients_stats.pivot_table(
index = ['child_status'], 
columns = 'debt_status',
values='debt',
aggfunc='count')
fmt = '{:.1%}'.format
child_debt_table['percent'] = (child_debt_table['была задолженность'] / (child_debt_table['была задолженность'] + child_debt_table['задолженности не было'])).map(fmt) #добавляем новый столбец- долю должников
child_debt_table

debt_status,была задолженность,задолженности не было,percent
child_status,,,
1-2 ребенка,647,6337,9.3%
детей нет,1063,13044,7.5%
многодетные,31,349,8.2%


**Вывод:**

Клиенты, у которых есть дети, менее охотно берут кредиты на какие-либо цели, однако доля должников у них выше, чем у клиентов без детей. Значит клиенты, у которых нет детей, более надежные. Самая рискованная группа - клиенты с 1-2 детьми.
</div>

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [17]:
family_debt_table = clients_stats.pivot_table(
index = ['family_status'], 
columns = 'debt_status',
values='debt',
aggfunc='count')
fmt = '{:.1%}'.format
family_debt_table['percent'] = (family_debt_table['была задолженность'] / (family_debt_table['была задолженность'] + family_debt_table['задолженности не было'])).map(fmt) #добавляем новый столбец- долю должников

family_debt_table

debt_status,была задолженность,задолженности не было,percent
family_status,,,
Не женат / не замужем,274,2536,9.8%
в разводе,85,1110,7.1%
вдовец / вдова,63,896,6.6%
гражданский брак,388,3775,9.3%
женат / замужем,931,11413,7.5%


**Вывод:**

Большинство  пользователей кредитами находятся в семейных отношениях. Самые ненадежные клиенты, с высокой долей должников- в статусе "Не женат / не замужем", самые надежные - клиенты категории "вдовец / вдова"

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [18]:
total_income_debt_table = clients_stats.pivot_table(
index = ['total_income_category'], 
columns = 'debt_status',
values='debt',
aggfunc='count')
fmt = '{:.1%}'.format
total_income_debt_table['percent'] = (total_income_debt_table['была задолженность'] / (total_income_debt_table['была задолженность'] + total_income_debt_table['задолженности не было'])).map(fmt) #добавляем новый столбец- долю должников

total_income_debt_table

debt_status,была задолженность,задолженности не было,percent
total_income_category,,,
высокий доход,358,4709,7.1%
низкий доход,354,4109,7.9%
средний доход,1029,10912,8.6%


**Вывод:**

Клиенты с высоким доходом чаще берут кредиты, нежели чем все остальные. Наиболее часто задолженность по кредитам имеют клиенты со средним доходом. Клиенты с низким доходом - самые надежные

**- Как разные цели кредита влияют на его возврат в срок?**

In [19]:
purpose_debt_table = clients_stats.pivot_table(
index = ['purpose_category'], 
columns = 'debt_status',
values='debt',
aggfunc='count')
fmt = '{:.1%}'.format
purpose_debt_table['percent'] = (purpose_debt_table['была задолженность'] / (purpose_debt_table['была задолженность'] + purpose_debt_table['задолженности не было'])).map(fmt) #добавляем новый столбец- долю должников

purpose_debt_table

debt_status,была задолженность,задолженности не было,percent
purpose_category,,,
операции с недвижимостью,782,10032,7.2%
получение образования,370,3644,9.2%
приобретение автомобиля,403,3905,9.4%
проведение свадьбы,186,2149,8.0%


**Вывод:**

Самое больше количество кредитов используется для проведения операций с недвижимостью: будь то покупка квартир, приобретение коммерческой неджвижимости или строительство дома. Получение образования и приобретение автомобиля - самые рискованные цели кредитования. 

## Общий вывод

Полученная в ходе исследования информация с данными клиентов банка была обработана: устранены недочеты в данных, связанные с отсутствующими и некоректными значениями, созданы категории отдельных данных, применен метод лемматизации данных. Использованные в работе сводные таблицы наглядно продемонстрировали наличие зависимости возврата кредита в срок от определенных факторов: семейного положения, наличия детей, уровня дохода и цели кредитования.

Сформированы профили самого надежного и самого рискованного клиента банка:

- Самый надежные клиенты банка- это вдовствующие люди, у которых нет детей, с месячным уровнем дохода до 100 000 руб., оформляющие кредит с целью проведения операций с неджвижимостью.

- Самая рискованная группа - это холостые люди, имеющие 1-2 детей, с месячным доходом от 100 до 200 тыс. руб., оформляющие кредит на приобретение автомобиля.

Исходя из полученных результатов можно построить скоринговую систему для оценки способности потенциального заёмщика к возврату кредита банку.
